This note book has 6 main steps:


1. Load the checkpoint
2. Score the file created by mBART by translating `english` to `hindi`
3. Score the file created by IndicTrans by translating `english` to `hindi`
4. Combine the logits from step 2, step 3, with the original sentence and the translated senences. This is also saved for further analysis
5. Apply `softmax` on the logits from step 2 and 3, resulting in `selecting` the better translations
6. Run sacreblue on the 
    a. mBART translations with reference to original 
    b. IndicTrans translations with reference to original 
    c. selected translations with reference to original 
    
    
If our hypothesis worked, we should have sacreblue from step 6c better than 6a or 6b

In [1]:
%cd indic-bert

/w266/w266/project/finetuning/indic-bert


In [2]:
import glob
import os

In [3]:
from fine_tune.modules import get_modules
from fine_tune.data import load_dataset
from fine_tune.data.examples import *
from fine_tune.modules import base

In [4]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

In [5]:
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [6]:
def _softmax(x1, x2, tog=0):
    a = np.exp([x1,x2])/np.sum(np.exp([x1,x2]))
    if tog == 1:
        return a[0]
    else:
        return a[1]

def apply_softmax(df, tog):
    return df.apply(lambda row: _softmax(row[0], row[1], tog), axis=1)

In [7]:
## Setting up parameters for the model
hparams = {
    'dataset': 'amrita-paraphrase-exact', 
    'model': 'ai4bharat/indic-bert', 
    'config_name': '', 
    'tokenizer_name': '', 
    'max_seq_length': 256, 
    'iglue_dir': '../indic-glue', 
    'overwrite_cache': True, 
    'cache_dir': '../ib-cache', 
    'fp16': False, 
    'fp16_opt_level': 'O1', 
    'n_gpu': 1, 
    'n_tpu_cores': 0, 
    'max_grad_norm': 1.0, 
    'do_train': True, 
    'do_predict': True, 
    'gradient_accumulation_steps': 1,
    'seed': 8, 
    'learning_rate': 2e-05, 
    'weight_decay': 0.0, 
    'adam_epsilon': 1e-08, 
    'warmup_steps': 0, 
    'num_train_epochs': 1, 
    'train_batch_size': 16, 
    'eval_batch_size': 32, 
    'labels': '', 
    'model_name_or_path': 'ai4bharat/indic-bert', 
    'data_dir': '../indic-glue/amrita-paraphrase-exact', 
    'mode': 'sequence-classification', 
    'output_mode': 'classification', #'classification', 
    'example_type': 'text', 
    'num_labels': 2
}

In [8]:
### Set up parameters for hindi
hparams['out_dir'] = '../outputs/8/'
hparams['lang'] = 'hi' 
hparams['train_lang'] = 'hi'
hparams['dev_lang'] = 'hi'
hparams['test_lang'] = 'hi'
hparams['output_dir'] = '{}amrita-paraphrase-exact/train-{}/model-ai4bharat-indic-bert'.format(hparams['out_dir'],hparams['lang'] ) 

In [9]:
config_name = hparams['config_name'] or hparams['model_name_or_path']
args = {'num_labels': hparams['num_labels']}
config = AutoConfig.from_pretrained(
    config_name,
    **args,
    cache_dir=hparams['cache_dir']
)

## Score mBART Hindi

In [10]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [11]:
!cp ../indic-glue/amrita-paraphrase-exact/hi/hi-bart-test.csv ../indic-glue/amrita-paraphrase-exact/hi/hi-test.csv 

In [12]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/hi/hi-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [13]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

### Load model from checkpoint

In [14]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

Set up a pytorch lighning Trainer

In [15]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../outputs/8/amrita-paraphrase-exact/train-hi/model-ai4bharat-indic-bert exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [16]:
trainer.test(trained_model)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Logits written to :../outputs/8/amrita-paraphrase-exact/train-hi/model-ai4bharat-indic-bert/test_results_hi.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.955,
 'avg_test_loss': 0.6586635708808899,
 'val_loss': 0.6586635708808899}
--------------------------------------------------------------------------------
{'avg_test_loss': 0.6586635708808899, 'val_loss': 0.6586635708808899, 'acc': 0.955}



## Load the scores for mBART translations.

In [17]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/8/amrita-paraphrase-exact/train-hi/model-ai4bharat-indic-bert/test_results_hi.csv'

In [18]:
mbart = pd.read_csv(hparams['results_file'], header=None)

In [19]:
mbart.shape

(1000, 4)

In [20]:
mbart.head(5)

,0,1,2,3
0,1.641388,-2.035115,0.0,1.0
1,-0.095410,0.146530,1.0,1.0
2,-0.093650,0.141301,1.0,1.0
3,-0.093835,0.141280,1.0,1.0
4,-0.095638,0.144878,1.0,1.0


## Score IndicTrans Hindi

In [28]:
! rm ../indic-glue/amrita-paraphrase-exact/cached*

In [29]:
!cp ../indic-glue/amrita-paraphrase-exact/hi/hi-indictrans-test.csv ../indic-glue/amrita-paraphrase-exact/hi/hi-test.csv 


In [30]:
## Check if you have the right file
mbart_test = pd.read_csv('../indic-glue/amrita-paraphrase-exact/hi/hi-test.csv', header=None)
mbart_test.shape

(1000, 3)

In [31]:
# module_name = 'text_classification'

text_classification_trainer = get_modules('text_classification')
text_classification_trainer

fine_tune.modules.text_classification.TextClassification

In [32]:
tc_trainer_instance = text_classification_trainer(hparams)
checkpoints = list(sorted(glob.glob(os.path.join(hparams['output_dir'], 'checkpointepoch=*.ckpt'), recursive=True)))
trained_model = tc_trainer_instance.load_from_checkpoint(checkpoints[-1])

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [33]:
trainer = base.create_trainer(trained_model, hparams) #BaseModule(hparams)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [34]:
trainer.test(trained_model)

Logits written to :../outputs/8/amrita-paraphrase-exact/train-hi/model-ai4bharat-indic-bert/test_results_hi.csv
--------------------------------------------------------------------------------
TEST RESULTS
{'acc': 0.976,
 'avg_test_loss': 0.6165903806686401,
 'val_loss': 0.6165903806686401}
--------------------------------------------------------------------------------
{'avg_test_loss': 0.6165903806686401, 'val_loss': 0.6165903806686401, 'acc': 0.976}



## Load the scores for IndicTrans translations.

In [35]:
hparams['results_file'] = '{}{}/train-{}/model-ai4bharat-indic-bert/test_results_{}.csv'.format(hparams['out_dir'],  hparams['dataset'] , hparams['lang'] , hparams['lang']) 
hparams['results_file']

'../outputs/8/amrita-paraphrase-exact/train-hi/model-ai4bharat-indic-bert/test_results_hi.csv'

In [36]:
indictrans = pd.read_csv(hparams['results_file'], header=None)

In [37]:
indictrans.shape

(1000, 4)

In [38]:
indictrans.head(5)

,0,1,2,3
0,-0.097524,0.144784,1.0,1.0
1,-0.093321,0.144864,1.0,1.0
2,-0.093762,0.140794,1.0,1.0
3,-0.095129,0.143009,1.0,1.0
4,-0.091816,0.139572,1.0,1.0


## Combine logits, original sentence, and translations

In [39]:
indic_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/hi/hi-indictrans-test.csv', header=None)

In [40]:
indic_orig.head()

,0,1,2
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,P
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व...",P
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,P
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,P
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब...",P


In [41]:
bart_orig = pd.read_csv('../indic-glue/amrita-paraphrase-exact/hi/hi-bart-test.csv', header=None)

In [42]:
bart_orig.head()

,0,1,2
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,P
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,P
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,P
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,P
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",P


In [43]:
combined = bart_orig[[0, 1]]

In [44]:
combined.columns =['Original_Sentence', 'mBART_Translated', ]

In [45]:
combined['IndicTrans_Translated'] = indic_orig[[1]]

In [46]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व..."
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब..."


In [47]:
# combined = pd.concat([bart_orig['sentence_1'], b_trans['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])

In [48]:
combined['mBART_NP_logits'] = mbart[[0]]
combined['mBART_P_logits'] = mbart[[1]]
combined['model_pred_mbart'] = ['NP' if x==0 else 'P' for x in mbart[2]]
combined['mbart_P_prob'] = apply_softmax(mbart, 0)
combined['mbart_NP_prob'] = apply_softmax(mbart, 1)

In [49]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_logits,mBART_P_logits,model_pred_mbart,mbart_P_prob,mbart_NP_prob
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,1.641388,-2.035115,NP,0.024686,0.975314
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व...",-0.095410,0.146530,P,0.560192,0.439808
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,-0.093650,0.141301,P,0.558469,0.441531
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,-0.093835,0.141280,P,0.558509,0.441491
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब...",-0.095638,0.144878,P,0.559841,0.440159


In [50]:
combined['IndicT_NP_logits'] = indictrans[[0]]
combined['IndicT_P_logits'] = indictrans[[1]]
combined['model_pred_IndicT'] = ['NP' if x==0 else 'P' for x in indictrans[2]]
combined['indicT_P_prob'] = apply_softmax(indictrans, 0)
combined['indicT_NP_prob'] = apply_softmax(indictrans, 1)

In [51]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_logits,mBART_P_logits,model_pred_mbart,mbart_P_prob,mbart_NP_prob,IndicT_NP_logits,IndicT_P_logits,model_pred_IndicT,indicT_P_prob,indicT_NP_prob
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,1.641388,-2.035115,NP,0.024686,0.975314,-0.097524,0.144784,P,0.560282,0.439718
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व...",-0.095410,0.146530,P,0.560192,0.439808,-0.093321,0.144864,P,0.559266,0.440734
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,-0.093650,0.141301,P,0.558469,0.441531,-0.093762,0.140794,P,0.558372,0.441628
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,-0.093835,0.141280,P,0.558509,0.441491,-0.095129,0.143009,P,0.559255,0.440745
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब...",-0.095638,0.144878,P,0.559841,0.440159,-0.091816,0.139572,P,0.557590,0.442410


In [54]:
def select_trasnlation(row):
    if row['indicT_P_prob']>=row['mbart_P_prob']:
        return row['IndicTrans_Translated']
    else:
        return row['mBART_Translated']

combined['selected_translation'] = combined.apply(lambda row : select_trasnlation(row), axis=1) 

In [55]:
combined['mBART_avg'] = combined.iloc[:,3:5].mean(axis=1)

In [56]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_logits,mBART_P_logits,model_pred_mbart,mbart_P_prob,mbart_NP_prob,IndicT_NP_logits,IndicT_P_logits,model_pred_IndicT,indicT_P_prob,indicT_NP_prob,selected_translation,mBART_avg
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,1.641388,-2.035115,NP,0.024686,0.975314,-0.097524,0.144784,P,0.560282,0.439718,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,-0.196864
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व...",-0.095410,0.146530,P,0.560192,0.439808,-0.093321,0.144864,P,0.559266,0.440734,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.025560
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,-0.093650,0.141301,P,0.558469,0.441531,-0.093762,0.140794,P,0.558372,0.441628,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.023825
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,-0.093835,0.141280,P,0.558509,0.441491,-0.095129,0.143009,P,0.559255,0.440745,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.023723
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब...",-0.095638,0.144878,P,0.559841,0.440159,-0.091816,0.139572,P,0.557590,0.442410,"इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.024620


In [57]:
combined['IT_avg'] = combined.iloc[:,5:7].mean(axis=1)

In [58]:
combined.head()

,Original_Sentence,mBART_Translated,IndicTrans_Translated,mBART_NP_logits,mBART_P_logits,model_pred_mbart,mbart_P_prob,mbart_NP_prob,IndicT_NP_logits,IndicT_P_logits,model_pred_IndicT,indicT_P_prob,indicT_NP_prob,selected_translation,mBART_avg,IT_avg
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,Prime Minister said Babasaheb Ambedkar has a k...,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,1.641388,-2.035115,NP,0.024686,0.975314,-0.097524,0.144784,P,0.560282,0.439718,प्रधानमंत्री ने कहा कि करोड़ों लोगों के दिलों ...,-0.196864,0.024686
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...,आज बीजापुर में इस समारोह को आयोजित करने के महत...,"""आज बीजापुर में इस कार्यक्रम के आयोजन के महत्व...",-0.095410,0.146530,P,0.560192,0.439808,-0.093321,0.144864,P,0.559266,0.440734,आज बीजापुर में इस समारोह को आयोजित करने के महत...,0.025560,0.560192
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,-0.093650,0.141301,P,0.558469,0.441531,-0.093762,0.140794,P,0.558372,0.441628,उन्होंने कहा कि लक्ष्य 2022 तक इस कार्य को पूर...,0.023825,0.558469
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,-0.093835,0.141280,P,0.558509,0.441491,-0.095129,0.143009,P,0.559255,0.440745,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.023723,0.558509
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...","""इस संदर्भ में उन्होंने जन धन खाते खोलने, गरीब...",-0.095638,0.144878,P,0.559841,0.440159,-0.091816,0.139572,P,0.557590,0.442410,"इस संदर्भ में उन्होंने जनधन खाते खोलने, गरीबों...",0.024620,0.559841


In [59]:
combined.to_csv('../combined.csv', sep='\t', encoding='utf-8')

In [60]:
np.savetxt('../original_sentence-hi.txt', combined['Original_Sentence'], fmt='%s')
np.savetxt('../mBART_translated-hi.txt', combined['mBART_Translated'], fmt='%s')
np.savetxt('../indictrans_translated-hi.txt', combined['IndicTrans_Translated'], fmt='%s')
np.savetxt('../selected_translation-hi.txt', combined['selected_translation'], fmt='%s')

In [61]:
!cp ../original_sentence-hi.txt ../../bleu_test/testing/indicTrans/
!cp ../mBART_translated-hi.txt  ../../bleu_test/testing/indicTrans/
!cp ../indictrans_translated-hi.txt  ../../bleu_test/testing/indicTrans/
!cp ../selected_translation-hi.txt  ../../bleu_test/testing/indicTrans/

# Measure the semantic similarity

In [62]:
# # Install the necessary libraries
# !pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# # Install fairseq from source
# !git clone https://github.com/pytorch/fairseq.git
# %cd fairseq
# # !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
# !pip install --editable ./
# %cd ..

In [63]:
# ! apt-get -y install jq

In [64]:
%cd ../../bleu_test/testing/indicTrans/

/w266/w266/project/bleu_test/testing/indicTrans


In [65]:
!sacrebleu original_sentence.txt -i mBART_translated-hi.txt -m bleu chrf ter

[
{
 "name": "BLEU",
 "score": 26.8,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.0.0",
 "verbose_score": "58.0/34.0/21.3/13.4 (BP = 0.980 ratio = 0.980 hyp_len = 17845 ref_len = 18206)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.0.0"
},
{
 "name": "chrF2",
 "score": 54.6,
 "signature": "nrefs:1|case:mixed|eff:yes|nc:6|nw:0|space:no|version:2.0.0",
 "nrefs": "1",
 "case": "mixed",
 "eff": "yes",
 "nc": "6",
 "nw": "0",
 "space": "no",
 "version": "2.0.0"
},
{
 "name": "TER",
 "score": 56.2,
 "signature": "nrefs:1|case:lc|tok:tercom|norm:no|punct:yes|asian:no|version:2.0.0",
 "nrefs": "1",
 "case": "lc",
 "tok": "tercom",
 "norm": "no",
 "punct": "yes",
 "asian": "no",
 "version": "2.0.0"
}
]


In [67]:
!sacrebleu original_sentence.txt -i mBART_translated-hi.txt -m bleu  --tokenize intl | jq -r .score
!sacrebleu original_sentence.txt -i mBART_translated-hi.txt -m chrf  --tokenize intl | jq -r .score
!sacrebleu original_sentence.txt -i mBART_translated-hi.txt -m ter   --tokenize intl | jq -r .score

28.1
54.6
56.2


In [70]:
!sacrebleu original_sentence.txt -i indictrans_translated-hi.txt -m bleu --tokenize intl | jq -r .score
!sacrebleu original_sentence.txt -i indictrans_translated-hi.txt -m chrf --tokenize intl | jq -r .score
!sacrebleu original_sentence.txt -i indictrans_translated-hi.txt -m  ter --tokenize intl | jq -r .score

38.7
63.5
48.4


In [71]:
!sacrebleu original_sentence.txt -i selected_translation-hi.txt -m bleu  --tokenize intl  | jq -r .score
!sacrebleu original_sentence.txt -i selected_translation-hi.txt -m chrf  --tokenize intl  | jq -r .score
!sacrebleu original_sentence.txt -i selected_translation-hi.txt -m   ter --tokenize intl  | jq -r .score


30.4
57
54.8


The End!